# A rudimentary implementation of the Histograms of Oriented Gradients image classification algorithm 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import hoglib as hog
from importlib import reload
import time
import sys
import pickle 
import os
import progressbar as pb

## Histogram generators
The following methods take raw images (with normalized dimensions) as input, and produce histograms suitable as descriptors in a learned classification algorithm, such as SVM

In [ ]:
def _convolve(image, conv_matrix):
    """
    DEPRECATED: use _detect_edge_vert and _detect_edge_hor insetead!
    Performs a convolution on a given image using a given 
    convolutional matrix. Edge overflow is considered to be zeros (black).
    """
    conv_matrix = conv_matrix/np.linalg.norm(conv_matrix)
    v_reach = int(conv_matrix.shape[0]/2)
    h_reach = int(conv_matrix.shape[1]/2)
    expanded_image = np.zeros(np.array(conv_matrix.shape)-1+image.shape)
    expanded_image[v_reach:-v_reach, h_reach:-v_reach]=image
    result = np.zeros(image.shape)
    for (i, j), _ in np.ndenumerate(image):
        result[i, j] = (conv_matrix*expanded_image[i:i+2*v_reach+1, j:j+2*h_reach+1]).sum()
    return result

def _detect_edge_vert(image):
    """
    Performs a convolution between the argument image and the vector [-1, 0, 1],
    thereby detecting vertical edges. The output is a convolution with the same
    dimensions as the input image. Edge overflow is treated as being 0.
    """
    #TODO: Optimize by using numpy.convolve
    output = np.zeros(image.shape)
    for i in range(image.shape[0]):
        for j in range(1,image.shape[1]-1):
            output[i,j] = image[i,j+1] - image[i,j-1]
        output[i,0] = image[i,1]
        output[i, -1] = -image[i, -2]
    return output

def _detect_edge_hor(image):
    """
    Performs a convolution between the argument image and the vector [-1; 0; 1],
    thereby detecting horizontal edges. The output is a convolution with the same
    dimensions as the input image. Edge overflow is treated as being 0.
    """
    #TODO: Optimize by using numpy.convolve
    output = np.zeros(image.shape)
    for j in range(image.shape[1]):
        for i in range(1,image.shape[0]-1):
            output[i,j] = image[i+1,j] - image[i-1,j]
        output[0, j] = -image[1,j]
        output[-1, j] = image[-2, j]
    return output

def _vec_to_deg_unsigned(vec):
    #TODO: Optimization, this is way slower than it should be!
    """
    Calculates the unsigned angle (0-180) of a given vector
    in relation to the positive x-axis.
    """
    if np.linalg.norm(vec) < 1e-10: 
        return 0
    else:
        deg = np.arccos(vec[0]/np.sqrt((vec[0]**2) + (vec[1]**2)))
        return deg*180/np.pi
    
def _rectify_angle(angle):
    """
    Converts an unsigned angle from the [-90, 90] format to the [0 180] format.
    """
    return angle if angle>0 else angle+180

def get_gradient_matrix(image):
    """
    Calculates the gradient matrix M of a given m-by-n input image.
    On exit, M is an m-by-n-by-2 tensor such that:
      M[:,:,0] represents the magnitude of the gradient vector
        for each pixel in the input image
      M[:,:,1] represents the angle of each vector
    """
    # Calculate gradients
    if len(image.shape) == 2:
        new_image = np.zeros((image.shape[0], image.shape[1], 3))
        new_image[:,:,0] = image
        image = new_image
        
    gr_h_r = _detect_edge_hor(image[:,:,0])
    gr_h_g = _detect_edge_hor(image[:,:,1])
    gr_h_b = _detect_edge_hor(image[:,:,2])
    gr_v_r = _detect_edge_vert(image[:,:,0])
    gr_v_g = _detect_edge_vert(image[:,:,1])
    gr_v_b = _detect_edge_vert(image[:,:,2])
    gr_h = np.maximum(np.maximum(gr_h_r, gr_h_g), gr_h_b)
    gr_v = np.maximum(np.maximum(gr_v_r, gr_v_g), gr_v_b)
    output = np.zeros([gr_v.shape[0], gr_v.shape[1], 2])
    # Save gradient magnitude
    output[:,:,0] = np.linalg.norm([gr_h,gr_v], axis=0)
    
    output[:,:,1] = np.angle(gr_v+gr_h*1j, deg = True)
    rectify = np.vectorize(_rectify_angle)
    output[:,:,1] = rectify(output[:,:,1])

    return output   #M

def _get_cos_bins(num_bins = 9):
#     TODO: Distribute vector across adjacent bins according to angle
    """
    DEPRECATED
    Returns a list of cosine values corresponding to an equidistant
    distribution of vector angles into num_bins categories.
    """
    cos_vals = np.cos(np.pi*np.linspace(0,num_bins,num_bins+1)/num_bins)
    return [np.sign(cos_val)*cos_val**2 for cos_val in cos_vals]

def _vec_to_bin(x, y, num_bins = 9, 
               cosine_bins_squared = None):
    """
    DEPRECATED
    Assigns a vector [x, y] to one of num_bins categories depending on its angle.
    Strongly advise to pre-calculate cosine_bins_squared if this function will be used
    repeatedly.
    """
    if (abs(x) < 0.1) and (abs(y) < 0.1): return 0 # discarding small gradients - yes or no?
    if cosine_bins_squared is None: cbs = _get_cos_bins(num_bins)
    else: cbs = cosine_bins_squared
    if y < 0: x, y = -x, -y
    cos_squared = np.sign(x)*(x**2)/(x**2+y**2)
    for i in range(num_bins): 
        if cbs[i] >= cos_squared > cbs[i+1]: return i
    if cos_squared == -1: return 8
    raise RuntimeError("Vector " + str([x,y]) + " could not be placed into any bin!")
    return 0    

def _get_bin(angle, num_bins): 
    """
    DEPRECATED
    Sorts an angle into one of a number of distinct categories, 
    or 'bins'. For example, 3 bins will each represent 60 degrees,
    so the angles 45, 65 and 175 degrees belong into bins
    0, 1 and 2 respectively
    """
    return int(round((angle/180)*(num_bins-1)))


def get_histogram(gradient_matrix, num_bins=9):
    """
    Takes a matrix of gradient vectors (see get_gradient_matrix()), 
    assigns them all into different bins and finally builds a histogram
    the x-axis of the histogram represents the possible bins
    (0, 1, 2,..., num_bins). Each gradient vector adds its magnitude
    to the appropriate bin.
    """    
    gm=gradient_matrix
    histogram = np.zeros(num_bins)
    for (i,j), _ in np.ndenumerate(gm[:,:,0]):
        histogram[int(gm[i,j,1])] += gm[i,j,0]
    return histogram


def crop_image(image, cell_height=8, cell_width=8):
    
    """
    DEPRECATED: Use hoglib.resize() instead
    Crops the bottom and right hand edges of an image so its
    height and width are multiples of cell_height and cell_width 
    respectively
    """
    if image.shape[0]%cell_height:
        image = image[:-(image.shape[0]%cell_height),:]
    if image.shape[1]%cell_width:
        image[:,:-(image.shape[1]%cell_width)]
    return image

def _grad_to_hist(mag, angle, num_bins=9):
    hist = np.zeros(num_bins)
    bin_width = 180/num_bins
    bin_centers = np.array(range(1,num_bins+1))*bin_width-bin_width/2
    
    #determine index of left-hand and right-hand bins
    if angle <= bin_centers[0]:
        idx_left = num_bins-1
        idx_right = 0
        factor = angle/bin_width + 0.5
    elif bin_centers[0] < angle < bin_centers[-1]:
        idx_left = bin_centers[bin_centers<angle].size-1
        idx_right = idx_left+1
        factor = (angle-bin_centers[idx_left])/bin_width
    else:
        idx_left = num_bins-1
        idx_right = 0
        factor = (angle-bin_centers[-1])/bin_width
    
    if not 0<factor<=1:
        raise RuntimeError(('Error determining appropriate bin for gradient with'
                            'angle {0}; magnitude {1} (adjacent bins'
                            '{2} and {3}, total bins {4}').format(
                                angle, mag, idx_left, idx_right, num_bins))
    
    #generate individual histogram
    hist[idx_left] = (1-factor)*mag
    hist[idx_right] = factor*mag
    return hist

def get_cell_histograms_8x8(grad_mat, num_cells_height, num_cells_width, num_bins=9):
    """
    Partitiones the image into a 32x32 grid of 8x8 pixel cells, then produces a 
    histogram for each cell (see get_histogram()).
    The output is a 32-by-32-by-num_bins tensor such that the slice [i,j,:] represents
    the histogram for the (i,j)-th grid tile of the source image.
    """
    #TODO: clean some of this hard code
    cell_histograms = np.zeros((num_cells_height, num_cells_width, num_bins))
    for i in range(num_cells_height):
        for j in range(num_cells_width):
            cell_histogram_current = np.zeros(num_bins)
            for i2 in range(8):
                for j2 in range(8):
                    cell_histogram_current = cell_histogram_current \
                                            + _grad_to_hist(
                                                grad_mat[i*8+i2, j*8+j2,0],
                                                grad_mat[i*8+i2, j*8+j2,1],
                                                num_bins)
            cell_histograms[i,j,:] = cell_histogram_current
    return cell_histograms

def L2_norm(v, eps):
    """Return the L2-norm with regularization"""
    return np.sqrt(v@v+eps)

def get_block_histograms_8x8(cell_histograms, num_bins=9):
    """
    Just check the paper idk
    """
    ch = cell_histograms
    height = ch.shape[0]-1
    width = ch.shape[1]-1
    cells = np.zeros((height, width, num_bins*4))
    for i in range(height):
        for j in range(width):
            current = np.zeros(num_bins*4)
            current[0*num_bins:1*num_bins] = ch[i,j,:]
            current[1*num_bins:2*num_bins] = ch[i+1,j,:]
            current[2*num_bins:3*num_bins] = ch[i,j+1,:]
            current[3*num_bins:4*num_bins] = ch[i+1,j+1,:]
            #normalize using L2-norm with regularization 
            cells[i,j] = current/L2_norm(current, 1e-5)
    return np.ravel(cells)

def show_histogram(image, cell_histograms, i=0, j=0, cell_height=8, cell_width=8):
    """
    Displays the relevant image, a single cell within it 
    (indexed by i and j), and finally the histogram relevant 
    to that cell.
    """
    h = cell_height; w = cell_width

    fig, (g1, g2, g3) = plt.subplots(3,1)
    g1.imshow(image, cmap='gray')
    g1.plot(    [j*w, (j+1)*w, (j+1)*w, j*w, j*w], 
                [i*h, i*h, (i+1)*h, (i+1)*h, i*h], 'r-')
    g2.imshow(image[i*h:(i+1)*h, j*w:(j+1)*w], cmap='gray')
    g2.plot(    [0, w-1, w-1, 0, 0], 
                [0, 0, h-1, h-1, 0], 'r--')
    g3.bar(range(0,9), cell_histograms[i,j,:], align='center', width=0.9)
    
def seconds_to_timestamp(elapsed):
    """
    Convert a time period (expressed by a number of seconds elapsed) to a timestamp and returns
    the number of hours, minutes and seconds elapsed.
    
    Example: 
        seconds_to_timestamp(3824) = [1, 3, 44]
    """
    hours = elapsed//3600
    minutes = (elapsed-3600*hours)//60
    seconds = (elapsed - 3600*hours - 60*minutes)
    return [hours, minutes, seconds]

#DEPRECATED
def get_descriptors(train_set):
    """
    DEPRECATED
    Generates HOG descriptors for all images in a given set. Input should be an array of grayscale, 256x256
    image matrices. Output will be an array of 31x31 numpy arrays, such that each element in each of the numpy
    arrays represents the histogram for one block of the corresponding image.
    """
    i = 0
    total = len(train_set)
    train_descriptors = [] 
    start = time.time()
    elapsed_previous = 0
    for image in train_set:
        current_grad = get_gradient_matrix(image)
        train_descriptors.append(get_block_histograms_8x8(get_cell_histograms_8x8(current_grad)))
        elapsed = time.time()-start
        frametime = elapsed-elapsed_previous
        elapsed_previous = elapsed
        elapsed_hours, elapsed_minutes, elapsed_seconds = seconds_to_timestamp(elapsed)
        remaining_hours, remaining_minutes, remaining_seconds = seconds_to_timestamp(elapsed*(total/max(1,i)-1))
        remaining = elapsed
        i = i+1
        progress = 100*i/total
        progress_bar= int(progress/5)
        sys.stdout.write('\r')
        sys.stdout.write("[%-20s] %d%% (%d/%d); elapsed: %dh, %dm, %.2fs; remaining: %dh, %dm, %.2fs; frametime: %.2f" % (
            '='*progress_bar, progress, i, total, 
            elapsed_hours, elapsed_minutes, elapsed_seconds,
            remaining_hours, remaining_minutes, remaining_seconds, frametime*1000))
        sys.stdout.flush()
    return train_descriptors

def get_descriptor(img):
    """Obtain the feature vector for a single image matrix"""
    num_cells_height = int(np.floor(img.shape[0]/8))
    num_cells_width = int(np.floor(img.shape[1]/8))
    return get_block_histograms_8x8(
                get_cell_histograms_8x8(
                    get_gradient_matrix(
                        img), num_cells_height, num_cells_width))

def _extract_descriptors(path, width, height):
    """
    Generates HOG descriptors for all images in a given set. Input should be the path to an image database, 
    such that the structure is split into a training and testing folder, and each of those is further split into
    a 'positive' and a 'negative' folder. 
    
    Output is four arrays of numpy.array items such that each item is a histogram representation of one of the images
    in the database. 
    """
    filenames = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    print('\nCommencing descriptor extraction for '+path)
    matrices = [resize(plt.imread(path+'/'+filename), height, width) for filename in pb.progressbar(filenames)]
    print('\nExtracted ' + str(len(matrices)) + ' image matrices')
    gradient_tensors = [get_gradient_matrix(image_matrix) for image_matrix in pb.progressbar(matrices)]
    print('\nFinished building gradients')
    matrices = None
    num_cells_height = int(np.floor(height/8))
    num_cells_width = int(np.floor(width/8))
    descriptors = [get_block_histograms_8x8(get_cell_histograms_8x8(gradient, num_cells_height, num_cells_width)) 
                   for gradient in pb.progressbar(gradient_tensors)]
    print('\nDescriptors generated for '+path)
    gradient_tensors = None
    return descriptors

def prepare_data(path, width, height):
    if path[-1] != '/':
        path = path+'/'
    train_desc_pos = _extract_descriptors(path+'train/positive', width, height)
    train_desc_neg = _extract_descriptors(path+'train/negative', width, height)
    test_desc_pos = _extract_descriptors(path+'test/positive', width, height)
    test_desc_neg = _extract_descriptors(path+'test/negative', width, height)
    
    train_set = np.array(train_desc_pos + train_desc_neg)
    train_key = np.append(np.ones(len(train_desc_pos)), np.zeros(len(train_desc_neg)))
    test_set = np.array(test_desc_pos + test_desc_neg)
    test_key = np.append(np.ones(len(test_desc_pos)), np.zeros(len(test_desc_neg)))
    
    with open(path+str(height)+'x'+str(width)+'_'+'train_set.p', 'wb') as f:
        pickle.dump(train_set, f)
    with open(path+str(height)+'x'+str(width)+'_'+'train_key.p', 'wb') as f:
        pickle.dump(train_key, f)
    with open(path+str(height)+'x'+str(width)+'_'+'test_set.p', 'wb') as f:
        pickle.dump(test_set, f)
    with open(path+str(height)+'x'+str(width)+'_'+'test_key.p', 'wb') as f:
        pickle.dump(test_key, f)
    
    return (train_set, train_key), (test_set, test_key)  

def retrieve_descriptors(path, width, height):
    with open(path+str(height)+'x'+str(width)+'_'+'train_set.p', 'rb') as f:
        trs=pickle.load(f)
    with open(path+str(height)+'x'+str(width)+'_'+'train_key.p', 'rb') as f:
        trk=pickle.load(f)
    with open(path+str(height)+'x'+str(width)+'_'+'test_set.p', 'rb') as f:
        tes=pickle.load(f)
    with open(path+str(height)+'x'+str(width)+'_'+'test_key.p', 'rb') as f:
        tek=pickle.load(f)
    return (trs,trk), (tes,tek)

## Visual Genome driver tools
The following methods are used for operating the [Visual Genome](https://visualgenome.org/) dataset after downloading it locally - specifically they locate images of cars in the dataset, then extract sub-images containing only the cars. 

In [ ]:
def load_metadata(path = 'Data/', fname = 'objects.json'):
    """
    Retrieves the Visual Genome database metadata - use before using any further functions!
    """
    with open(path+fname, 'r') as f:
        return json.load(f)
    #TODO: except IOException, possibly automatically download the metadata

def has_car(json_obj = None):
    """
    json_obj should be an element of the json list retrieved by load_metadata()
    Returns True if the given image containes a 'car' object
    """
    if json_obj is None:
        json_obj = load_metadata();
    obj_list = [name for names in [item['names'] for item in json_obj['objects']] for name in names]
    return ('car' in obj_list)

def get_car_box(json_obj):
    """
    Retrieves a list of all car box coordinates for a given object. Return type is a list even
    if there is only a single box to return.
    
    Parameters:
    
    json_obj: a jason object as found in the Visual Genome database metadata, containing at least one object
        marked 'car'. json_object is an element of the list returned by load_metadata()
        
    Output:
    
    Output is a list of dictionaries with 5 keys: id, x, y, h, and w. Each dictionary represents a different car
    bounding box in the image associated with the argument.
        id: the unique identifier associated with each image in the database. Also found in the names of 
            each image file
        x, y: the row- and column- coordinates for a car bounding box. x is the number of rows from the top,
            and y is the number of columns from the left.
        h, w: the height and width of the car bounding box. 
    """
    car_id = -1
    car_list = [obj for obj in json_obj['objects'] if 'car' in obj['names']]
    output = []
    for obj in car_list:
        output.append(
                {
                'id' : json_obj['image_id'],
                'x' : obj['x'],
                'y' : obj['y'],
                'w' : obj['w'],
                'h' : obj['h']
                }
        )
    # Check if output is empty
    if not output:
        raise RuntimeError("No cars found in argument - json_obj should contain at least one car object (car box)")
    return output        

def extract_car_section(coordinates):
    """
    Retrieves a sub-section of the image matrix starting at column y, row x 
    and extending h rows down and w rows to the right
    
    Parameters:
        coordinates: coordinates for the car bounding box (id, x, y, h, w). See get_car_box() for more details.
    
    Output:
        returns a subsection of the parent image containing only the car bounding box (format readable by 
        matplotlib.pyplot.imshow)
    """
    id = coordinates['id']
    c = coordinates
    x, y, w, h = c['x'], c['y'], c['w'], c['h'] 
    im = plt.imread('Data/VG_100K/' + str(id) + '.jpg')
    return im[y:y+h, x:x+w]

def get_car_boxes(obj_list = None):
    """Retrieves a list of box-coordinates(id, x, y, height, width) for all car items in a list of objects."""
    if obj_list is None: 
        print("No object list provided - loading database metadata automatically. This may take a few seconds.\n")
        obj_list = load_metadata()
    output = []
    for item in [image for image in obj_list if has_car(image)]: output = output + get_car_box(item)
    return output

## General-purpose image processing
The following methods are used for slightly pre-processing the image data before feeding it into the HOG exctractor. Namely, they determine whether an image has suitable dimensions, then normalize the image by converting to grayscale, then resizing to a pre-determined size.

In [ ]:
def imread_gs(fname):
    img = plt.imread(fname)
    img = np.sum(img, axis = 2)/3
    max_val = np.max(img)
    return img/max_val
    
def show(img):
    plt.imshow(img, cmap = 'gray')

def _get_distorted_unity(height, width, fill=True): 
    """
    Generates a 'pseudo-diagonal' identity matrix, used for transforming dimensions of other matrices.

    Parameters:
    
    height, width : dimensions of the output matrix
    fill: 
        If True, a standard identity matrix is expanded by adding rows or columns containing ones, then normalizing
        each row or column. For example:

            [1 0 0]
            [1 0 0]
            [0 1 0]
            [0 1 0]
            [0 0 1]
            [0 0 1]

        If False, a standard identity matrix is expanded by adding empty rows or columns. For example:
            [1 0 0]
            [0 0 0]
            [0 1 0]
            [0 0 0]
            [0 0 1]
            [0 0 0]
    """
    M = max(height, width)
    m = min(height, width)
    if not fill: M, m = m, M
    u = np.zeros((M, m))
    k = m/M
    for i in range(M):
        u[i, int(i*k)] = 1
    if not fill: u = np.transpose(u)
    if height < width: u = np.transpose(u)
    return u


def transform_height(image, height):
    tr_mat = _get_distorted_unity(height, image.shape[0], height > image.shape[0])
    return tr_mat@image


def transform_width(image, width):
    tr_mat = _get_distorted_unity(image.shape[1], width, width > image.shape[1])
    return image@tr_mat

def suitable_dims(coordinates, min_h = 128, min_w = 128, min_stretch = 0.5, max_stretch = 2):
    """
    Determines wether a car image, specified by the coordinates variable, has dimensions suitable for training. 
    Images that are too small, or have an extreme ratio between their height and width, are considered unsuitable and the function returns false.
    
    Parameters:
    
    coordinates: see get_car_box()
    min_h = minimum suitable height (if none, set as 0)
    min_w = minimum suitable width (if none, set as 0)
    min_stretch = minimum suitable value for the ratio between height and width
    max_stretch = maximum suitable value for the ratio between height and width
    """
    c_h = coordinates['h']
    c_w = coordinates['w']
    if (min_h > 0) and (c_h < min_h): return False
    if (min_w>0) and (c_w < min_w): return False
    if (min_stretch > 0) and (c_h/c_w < min_stretch): return False
    if (max_stretch > 0) and (c_h/c_w > max_stretch): return False
    return True

def imread(fpath):
    """
    Read an image using matplotlib.pyplot.imread, then normalize it to have 3 color channels.
    """
    img = plt.imread(fpath)
    if len(img.shape) == 2:
        img_new = np.zeros((img.shape[0], img.shape[1], 3))
        img_new[:,:,0] = img
        return img_new
    return img[:,:,:3]
        

def resize(image, height, width):
    """
    Alters image size by either copying or deleting rows and columns of pixels as necassary
    """
    #Convert to grayscale if necessary
    if len(image.shape)>2:
        image_new = np.zeros((height,width, image.shape[2]))
        for i in range(image.shape[2]):
            image_new[:,:,i] = transform_height(transform_width(image[:,:,i], width), height)
    else: 
        image_new=transform_height(transform_width(image, width), height)
    return image_new/np.max([np.max(image_new), 1])